In [48]:
import pandas as pd

# Load the uploaded CSV files
projections_path = "C:/Users/PC/desktop/바탕 화면/archive/indiana_projections.csv"
reports_path ="C:/Users/PC/desktop/바탕 화면/archive/indiana_reports.csv"


projections_df = pd.read_csv(projections_path)
reports_df = pd.read_csv(reports_path)

# 데이터 구조 확인
projections_df_info = projections_df.info()
reports_df_info = reports_df.info()

projections_head = projections_df.head()
reports_head = reports_df.head()

(projections_df_info, reports_df_info, projections_head, reports_head)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7466 entries, 0 to 7465
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         7466 non-null   int64 
 1   filename    7466 non-null   object
 2   projection  7466 non-null   object
dtypes: int64(1), object(2)
memory usage: 175.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3851 entries, 0 to 3850
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         3851 non-null   int64 
 1   MeSH        3851 non-null   object
 2   Problems    3851 non-null   object
 3   image       3851 non-null   object
 4   indication  3765 non-null   object
 5   comparison  2685 non-null   object
 6   findings    3337 non-null   object
 7   impression  3820 non-null   object
dtypes: int64(1), object(7)
memory usage: 240.8+ KB


(None,
 None,
    uid                filename projection
 0    1  1_IM-0001-4001.dcm.png    Frontal
 1    1  1_IM-0001-3001.dcm.png    Lateral
 2    2  2_IM-0652-1001.dcm.png    Frontal
 3    2  2_IM-0652-2001.dcm.png    Lateral
 4    3  3_IM-1384-1001.dcm.png    Frontal,
    uid                                               MeSH  \
 0    1                                             normal   
 1    2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
 2    3                                             normal   
 3    4  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4    5  Osteophyte/thoracic vertebrae/multiple/small;T...   
 
                                             Problems  \
 0                                             normal   
 1                      Cardiomegaly;Pulmonary Artery   
 2                                             normal   
 3  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4                         Osteophyte;Thickening;Lung   
 
         

In [50]:
# Filter for Frontal images
frontal_projections = projections_df[projections_df['projection'] == 'Frontal']

# Merge with reports based on 'uid'
merged_df = pd.merge(frontal_projections, reports_df, on='uid')

# Select 'Problems' instead of 'MeSH' for labels
filtered_data_with_problems = merged_df[['filename', 'Problems']]

# Check the distribution of 'Problems'
filtered_data_with_problems.head(), filtered_data_with_problems['Problems'].value_counts()


(                    filename  \
 0     1_IM-0001-4001.dcm.png   
 1     2_IM-0652-1001.dcm.png   
 2     3_IM-1384-1001.dcm.png   
 3     4_IM-2050-1001.dcm.png   
 4  5_IM-2117-1003002.dcm.png   
 
                                             Problems  
 0                                             normal  
 1                      Cardiomegaly;Pulmonary Artery  
 2                                             normal  
 3  Pulmonary Disease, Chronic Obstructive;Bullous...  
 4                         Osteophyte;Thickening;Lung  ,
 Problems
 normal                                                                                                                             1387
 No Indexing                                                                                                                          91
 Lung                                                                                                                                 86
 Calcified Granuloma                      

In [52]:
# Extract the first label from the 'Problems' field
filtered_data_with_problems['Primary_Label'] = filtered_data_with_problems['Problems'].str.split(';').str[0]

# Check the label distribution for the primary label
filtered_data_with_primary_label = filtered_data_with_problems[['filename', 'Primary_Label']]
label_distribution = filtered_data_with_primary_label['Primary_Label'].value_counts()

# Display the updated dataset and the label distribution
filtered_data_with_primary_label.head(), label_distribution


C:\Users\PC\AppData\Local\Temp\ipykernel_16616\815716579.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data_with_problems['Primary_Label'] = filtered_data_with_problems['Problems'].str.split(';').str[0]


(                    filename                           Primary_Label
 0     1_IM-0001-4001.dcm.png                                  normal
 1     2_IM-0652-1001.dcm.png                            Cardiomegaly
 2     3_IM-1384-1001.dcm.png                                  normal
 3     4_IM-2050-1001.dcm.png  Pulmonary Disease, Chronic Obstructive
 4  5_IM-2117-1003002.dcm.png                              Osteophyte,
 Primary_Label
 normal                         1387
 Lung                            340
 Cardiomegaly                    240
 Opacity                         189
 Calcified Granuloma             172
                                ... 
 Pneumoperitoneum                  1
 Contrast Media                    1
 Thoracic vertebrae                1
 Subcutaneous Emphysema            1
 Lung Diseases, Interstitial       1
 Name: count, Length: 92, dtype: int64)

In [54]:
# Prepare filenames and labels for single-label classification
filenames_single_label = filtered_data_with_primary_label['filename'].values
labels_single_label = filtered_data_with_primary_label['Primary_Label'].values

# Encode labels to integers using sklearn's LabelEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels_single_label)

# Map encoded labels back to their original names for reference
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

# Split data into train, validation, and test sets
X_train_filenames, X_temp_filenames, y_train_labels, y_temp_labels = train_test_split(
    filenames_single_label, encoded_labels, test_size=0.3, random_state=42
)
X_val_filenames, X_test_filenames, y_val_labels, y_test_labels = train_test_split(
    X_temp_filenames, y_temp_labels, test_size=0.5, random_state=42
)

# Display dataset information
len(X_train_filenames), len(X_val_filenames), len(X_test_filenames), label_mapping


(2672,
 573,
 573,
 {'Airspace Disease': 0,
  'Aorta': 1,
  'Aorta, Thoracic': 2,
  'Aortic Aneurysm': 3,
  'Arthritis': 4,
  'Atherosclerosis': 5,
  'Bone Diseases, Metabolic': 6,
  'Breast Implants': 7,
  'Bullous Emphysema': 8,
  'Calcified Granuloma': 9,
  'Calcinosis': 10,
  'Cardiac Shadow': 11,
  'Cardiomegaly': 12,
  'Catheters, Indwelling': 13,
  'Cavitation': 14,
  'Cervical Vertebrae': 15,
  'Cicatrix': 16,
  'Consolidation': 17,
  'Contrast Media': 18,
  'Costophrenic Angle': 19,
  'Deformity': 20,
  'Density': 21,
  'Diaphragm': 22,
  'Diaphragmatic Eventration': 23,
  'Emphysema': 24,
  'Epicardial Fat': 25,
  'Foreign Bodies': 26,
  'Fractures, Bone': 27,
  'Funnel Chest': 28,
  'Granuloma': 29,
  'Granulomatous Disease': 30,
  'Heart': 31,
  'Heart Atria': 32,
  'Heart Ventricles': 33,
  'Hernia, Diaphragmatic': 34,
  'Hernia, Hiatal': 35,
  'Humerus': 36,
  'Hydropneumothorax': 37,
  'Hyperostosis, Diffuse Idiopathic Skeletal': 38,
  'Hypovolemia': 39,
  'Implanted Med

In [56]:
from sklearn.model_selection import train_test_split

# Split data into train, validation, and test sets
X_train_filenames, X_temp_filenames, y_train_labels, y_temp_labels = train_test_split(
    filenames_single_label, encoded_labels, test_size=0.3, random_state=42
)
X_val_filenames, X_test_filenames, y_val_labels, y_test_labels = train_test_split(
    X_temp_filenames, y_temp_labels, test_size=0.5, random_state=42
)

# Display dataset information: train, validation, test sizes and label mapping
len(X_train_filenames), len(X_val_filenames), len(X_test_filenames), label_mapping


(2672,
 573,
 573,
 {'Airspace Disease': 0,
  'Aorta': 1,
  'Aorta, Thoracic': 2,
  'Aortic Aneurysm': 3,
  'Arthritis': 4,
  'Atherosclerosis': 5,
  'Bone Diseases, Metabolic': 6,
  'Breast Implants': 7,
  'Bullous Emphysema': 8,
  'Calcified Granuloma': 9,
  'Calcinosis': 10,
  'Cardiac Shadow': 11,
  'Cardiomegaly': 12,
  'Catheters, Indwelling': 13,
  'Cavitation': 14,
  'Cervical Vertebrae': 15,
  'Cicatrix': 16,
  'Consolidation': 17,
  'Contrast Media': 18,
  'Costophrenic Angle': 19,
  'Deformity': 20,
  'Density': 21,
  'Diaphragm': 22,
  'Diaphragmatic Eventration': 23,
  'Emphysema': 24,
  'Epicardial Fat': 25,
  'Foreign Bodies': 26,
  'Fractures, Bone': 27,
  'Funnel Chest': 28,
  'Granuloma': 29,
  'Granulomatous Disease': 30,
  'Heart': 31,
  'Heart Atria': 32,
  'Heart Ventricles': 33,
  'Hernia, Diaphragmatic': 34,
  'Hernia, Hiatal': 35,
  'Humerus': 36,
  'Hydropneumothorax': 37,
  'Hyperostosis, Diffuse Idiopathic Skeletal': 38,
  'Hypovolemia': 39,
  'Implanted Med

In [64]:
# Load and preprocess images for single-label classification
def load_images_from_filenames(filenames, directory, target_size):
    images = []
    for filename in filenames:
        img_path = os.path.join(directory, filename)
        try:
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
            images.append(np.zeros((*target_size, 3)))
    return np.array(images)

# Image parameters
img_size = (224, 224)

# Load and preprocess images for train, validation, and test sets
X_train_images = load_images_from_filenames(X_train_filenames, image_dir, img_size)
X_val_images = load_images_from_filenames(X_val_filenames, image_dir, img_size)
X_test_images = load_images_from_filenames(X_test_filenames, image_dir, img_size)

# Normalize image data
X_train_images = X_train_images / 255.0
X_val_images = X_val_images / 255.0
X_test_images = X_test_images / 255.0

# Check shapes of the datasets
X_train_images.shape, X_val_images.shape, X_test_images.shape


((2672, 224, 224, 3), (573, 224, 224, 3), (573, 224, 224, 3))

In [65]:
# Define the image directory provided by the user
image_dir = r'C:/Users/PC/desktop/바탕 화면/images_normalized'

# Load and preprocess images for train, validation, and test sets
X_train_images = load_images_from_filenames(X_train_filenames, image_dir, img_size)
X_val_images = load_images_from_filenames(X_val_filenames, image_dir, img_size)
X_test_images = load_images_from_filenames(X_test_filenames, image_dir, img_size)

# Normalize image data
X_train_images = X_train_images / 255.0
X_val_images = X_val_images / 255.0
X_test_images = X_test_images / 255.0

# Check shapes of the datasets
X_train_images.shape, X_val_images.shape, X_test_images.shape


((2672, 224, 224, 3), (573, 224, 224, 3), (573, 224, 224, 3))

In [136]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.data import AUTOTUNE

# Limit TensorFlow to use only 2 CPU threads
import os
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["TF_NUM_INTRAOP_THREADS"] = "2"
os.environ["TF_NUM_INTEROP_THREADS"] = "2"

# Preprocess function
def preprocess(image, label):
    image = tf.image.resize(image, [128, 128]) / 255.0  # Resize and normalize
    return image, label

# Create datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_images, y_train_labels))
train_dataset = train_dataset.map(preprocess, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(64).prefetch(AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val_images, y_val_labels))
val_dataset = val_dataset.map(preprocess, num_parallel_calls=AUTOTUNE).batch(64).prefetch(AUTOTUNE)

# Define model
base_model = MobileNetV2(weights=None, include_top=False, input_shape=(128, 128, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_mapping), activation='softmax')  # Replace `len(label_mapping)` with number of classes
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.1930 - loss: 4.0345 - val_accuracy: 0.3543 - val_loss: 4.4927
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.3472 - loss: 3.1807 - val_accuracy: 0.3543 - val_loss: 4.4694
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.3582 - loss: 3.0739 - val_accuracy: 0.3543 - val_loss: 4.4418
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.3600 - loss: 3.0303 - val_accuracy: 0.3543 - val_loss: 4.4128
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.3530 - loss: 3.0569 - val_accuracy: 0.3543 - val_loss: 4.3853
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.3455 - loss: 2.9891 - val_accuracy: 0.3543 - val_loss: 4.3566
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.3548 - loss: 2.9846 - val_accuracy: 0.3543 - val_loss: 4.3177
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.3677 - loss: 2.9275 - val_accuracy: 0.3543 - val_loss:

In [155]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# 실제 테스트 데이터에 사용된 클래스 확인
used_classes = np.unique(y_test_labels)

# 성능 지표 계산
accuracy = accuracy_score(y_test_labels, y_pred_labels)
precision = precision_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)
recall = recall_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)
f1 = f1_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)

# 분류 보고서 생성
report = classification_report(
    y_test_labels,
    y_pred_labels,
    labels=used_classes,
    target_names=[label_encoder.classes_[i] for i in used_classes],
    zero_division=0
)

# 결과 출력
print("=== 성능 지표 ===")
print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-Score  : {f1:.4f}")



=== 성능 지표 ===
Accuracy  : 0.3578
Precision : 0.1280
Recall    : 0.3578
F1-Score  : 0.1885
